# 傷検出AI開発 - シンプル版（Teachable Machine不要）

このノートブックでは、TensorFlow/Kerasを直接使って傷検出AIを作成します。
より理解しやすく、カスタマイズも簡単な実装です。

## なぜこちらの方法も学ぶ？
- AIの仕組みをより深く理解できる
- 自由にモデルをカスタマイズできる
- 実際の開発現場に近い体験ができる

In [ ]:
# 必要なライブラリのインストール
!pip install tensorflow opencv-python pillow numpy matplotlib gradio scikit-learn

In [ ]:
# ライブラリのインポート
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import drive
from google.colab import files
import gradio as gr
from datetime import datetime
import pandas as pd

# 日本語フォントの設定
!apt-get -y install fonts-ipafont-gothic
plt.rcParams['font.family'] = 'IPAGothic'

print("✅ セットアップ完了！")

## 1. Google Driveの準備とデータ収集

In [ ]:
# Google Driveをマウント
drive.mount('/content/drive')

# プロジェクトフォルダの作成
project_path = '/content/drive/MyDrive/damage_detection_simple'
os.makedirs(project_path, exist_ok=True)
os.makedirs(f'{project_path}/dataset/good', exist_ok=True)
os.makedirs(f'{project_path}/dataset/bad', exist_ok=True)
os.makedirs(f'{project_path}/models', exist_ok=True)
os.makedirs(f'{project_path}/results', exist_ok=True)

print("📁 フォルダ構造:")
print(f"{project_path}/")
print("├── dataset/")
print("│   ├── good/  # 良品画像")
print("│   └── bad/   # 不良品画像")
print("├── models/    # 保存されたモデル")
print("└── results/   # 結果とレポート")

In [ ]:
# データアップロード用の関数
def upload_images(category='good'):
    """
    画像をアップロードしてデータセットフォルダに保存
    """
    print(f"📤 {'良品' if category == 'good' else '不良品'}の画像をアップロードしてください")
    uploaded = files.upload()
    
    count = 0
    for filename in uploaded.keys():
        # 保存先パス
        save_path = f"{project_path}/dataset/{category}/{filename}"
        
        # 画像を開いて保存（形式を統一）
        img = Image.open(filename)
        img = img.convert('RGB')  # RGBに変換
        img.save(save_path, 'JPEG')
        
        count += 1
        print(f"✓ {filename} を保存しました")
    
    print(f"\n✅ {count}枚の画像を {category} フォルダに保存しました")
    return count

# 使用例（コメントを外して実行）
# upload_images('good')  # 良品画像をアップロード
# upload_images('bad')   # 不良品画像をアップロード

In [ ]:
# データセットの確認
def check_dataset():
    """
    データセットの状態を確認
    """
    good_images = os.listdir(f"{project_path}/dataset/good")
    bad_images = os.listdir(f"{project_path}/dataset/bad")
    
    good_count = len([f for f in good_images if f.endswith(('.jpg', '.jpeg', '.png'))])
    bad_count = len([f for f in bad_images if f.endswith(('.jpg', '.jpeg', '.png'))])
    
    # グラフで表示
    plt.figure(figsize=(8, 5))
    categories = ['良品', '不良品']
    counts = [good_count, bad_count]
    colors = ['green', 'red']
    
    bars = plt.bar(categories, counts, color=colors)
    plt.title('データセットの内訳', fontsize=16)
    plt.ylabel('画像数', fontsize=12)
    
    # 数値を表示
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                str(count), ha='center', va='bottom', fontsize=12)
    
    plt.show()
    
    print(f"\n📊 データセット統計")
    print(f"良品: {good_count}枚")
    print(f"不良品: {bad_count}枚")
    print(f"合計: {good_count + bad_count}枚")
    
    if good_count < 20 or bad_count < 20:
        print("\n⚠️ 警告: 各カテゴリ20枚以上の画像を推奨します")
    
    return good_count, bad_count

# データセットを確認
check_dataset()

## 2. シンプルなCNNモデルの構築

畳み込みニューラルネットワーク（CNN）を使って、画像から特徴を学習します。

In [ ]:
# データの前処理とデータセット作成
def create_dataset(data_dir, image_size=(128, 128), batch_size=16):
    """
    画像データセットを作成
    """
    # データ拡張の設定（学習データの多様性を増やす）
    data_augmentation = keras.Sequential([
        layers.RandomFlip("horizontal"),  # 左右反転
        layers.RandomRotation(0.1),        # 回転
        layers.RandomZoom(0.1),            # ズーム
    ])
    
    # データセットを作成
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,  # 20%を検証用に
        subset="training",
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='binary'  # 2クラス分類
    )
    
    val_dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='binary'
    )
    
    # クラス名を取得
    class_names = dataset.class_names
    print(f"クラス: {class_names}")
    
    # データ拡張を適用
    dataset = dataset.map(lambda x, y: (data_augmentation(x), y))
    
    # パフォーマンス最適化
    AUTOTUNE = tf.data.AUTOTUNE
    dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
    val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
    
    return dataset, val_dataset, class_names

# データセットの準備
train_ds, val_ds, class_names = create_dataset(f"{project_path}/dataset")
print(f"\n✅ データセット準備完了")
print(f"クラス: {class_names}")

In [ ]:
# サンプル画像の表示
def show_sample_images(dataset, class_names):
    """
    データセットからサンプル画像を表示
    """
    plt.figure(figsize=(10, 8))
    
    # 1バッチ取得
    for images, labels in dataset.take(1):
        for i in range(min(9, len(images))):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            
            # ラベルを表示（0: bad, 1: good）
            label_idx = int(labels[i])
            label_name = class_names[label_idx]
            color = 'green' if label_name == 'good' else 'red'
            
            plt.title(f'{label_name}', color=color)
            plt.axis("off")
    
    plt.suptitle('データセットのサンプル画像', fontsize=16)
    plt.tight_layout()
    plt.show()

# サンプル表示
show_sample_images(train_ds, class_names)

In [ ]:
# シンプルなCNNモデルの構築
def create_simple_cnn(input_shape=(128, 128, 3)):
    """
    シンプルで理解しやすいCNNモデル
    """
    model = keras.Sequential([
        # 入力の正規化
        layers.Rescaling(1./255, input_shape=input_shape),
        
        # 畳み込み層1（特徴を抽出）
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        
        # 畳み込み層2（より複雑な特徴を抽出）
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        
        # 畳み込み層3（さらに複雑な特徴を抽出）
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        
        # ドロップアウト（過学習防止）
        layers.Dropout(0.5),
        
        # 全結合層
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # 2クラス分類
    ])
    
    # モデルのコンパイル
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# モデルの作成
model = create_simple_cnn()

# モデルの構造を表示
model.summary()

## 3. モデルの学習

In [ ]:
# 学習の実行
def train_model(model, train_ds, val_ds, epochs=20):
    """
    モデルの学習を実行
    """
    # コールバックの設定
    callbacks = [
        # 改善が見られない場合は学習率を下げる
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=0.00001,
            verbose=1
        ),
        # 改善が見られない場合は早期終了
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
    ]
    
    print("🚀 学習を開始します...")
    
    # 学習実行
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks
    )
    
    print("\n✅ 学習完了！")
    
    return history

# 学習実行（エポック数は調整可能）
history = train_model(model, train_ds, val_ds, epochs=20)

In [ ]:
# 学習結果の可視化
def plot_training_history(history):
    """
    学習履歴をグラフで表示
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # 精度のグラフ
    ax1.plot(history.history['accuracy'], label='学習データ')
    ax1.plot(history.history['val_accuracy'], label='検証データ')
    ax1.set_title('モデルの精度', fontsize=14)
    ax1.set_xlabel('エポック')
    ax1.set_ylabel('精度')
    ax1.legend()
    ax1.grid(True)
    
    # 損失のグラフ
    ax2.plot(history.history['loss'], label='学習データ')
    ax2.plot(history.history['val_loss'], label='検証データ')
    ax2.set_title('モデルの損失', fontsize=14)
    ax2.set_xlabel('エポック')
    ax2.set_ylabel('損失')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # 最終的な精度を表示
    final_accuracy = history.history['val_accuracy'][-1]
    print(f"\n📊 最終的な検証精度: {final_accuracy:.1%}")
    
    if final_accuracy < 0.8:
        print("💡 ヒント: データを増やすか、学習エポック数を増やしてみてください")
    else:
        print("🎉 良い精度が出ています！")

# 学習結果を表示
plot_training_history(history)

## 4. モデルの保存と読み込み

In [ ]:
# モデルの保存
def save_model(model, model_name="damage_detector"):
    """
    モデルを保存
    """
    # 保存パス
    save_path = f"{project_path}/models/{model_name}.h5"
    
    # モデルを保存
    model.save(save_path)
    print(f"💾 モデルを保存しました: {save_path}")
    
    # クラス名も保存
    with open(f"{project_path}/models/class_names.txt", 'w') as f:
        for name in class_names:
            f.write(f"{name}\n")
    
    return save_path

# モデルを保存
model_path = save_model(model)

## 5. 予測機能の実装

In [ ]:
# 予測クラスの実装
class SimpleDamageDetector:
    """
    シンプルな傷検出器
    """
    def __init__(self, model_path, image_size=(128, 128)):
        self.model = tf.keras.models.load_model(model_path)
        self.image_size = image_size
        self.history = []
        
        # クラス名の読み込み
        class_names_path = model_path.replace('.h5', '').replace('damage_detector', 'class_names.txt')
        try:
            with open(class_names_path, 'r') as f:
                self.class_names = [line.strip() for line in f.readlines()]
        except:
            self.class_names = ['bad', 'good']  # デフォルト
    
    def preprocess_image(self, image):
        """
        画像の前処理
        """
        if isinstance(image, str):
            # ファイルパスの場合
            img = Image.open(image)
        else:
            # numpy配列の場合
            img = Image.fromarray(image)
        
        # RGBに変換してリサイズ
        img = img.convert('RGB')
        img = img.resize(self.image_size)
        
        # numpy配列に変換
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)
        
        return img_array, img
    
    def predict(self, image):
        """
        画像から傷を検出
        """
        # 前処理
        img_array, original_img = self.preprocess_image(image)
        
        # 予測
        prediction = self.model.predict(img_array, verbose=0)[0][0]
        
        # 結果を整理（0に近い=bad, 1に近い=good）
        if prediction < 0.5:
            result = 'bad'
            confidence = 1 - prediction
        else:
            result = 'good'
            confidence = prediction
        
        # 履歴に追加
        self.history.append({
            'timestamp': datetime.now(),
            'result': result,
            'confidence': float(confidence)
        })
        
        return result, confidence, original_img
    
    def analyze(self, image):
        """
        詳細な分析結果を返す
        """
        result, confidence, img = self.predict(image)
        
        # 判定基準
        if result == 'bad' and confidence > 0.8:
            status = "不良品"
            action = "廃棄または再加工を推奨"
            emoji = "❌"
        elif result == 'bad' and confidence > 0.6:
            status = "要確認"
            action = "目視での再確認を推奨"
            emoji = "⚠️"
        else:
            status = "良品"
            action = "次工程へ進めて問題ありません"
            emoji = "✅"
        
        return {
            'status': status,
            'confidence': confidence,
            'action': action,
            'emoji': emoji,
            'image': img
        }

# 検出器を初期化
detector = SimpleDamageDetector(model_path)
print("✅ 検出器の準備完了！")

In [ ]:
# テスト実行
print("🖼️ テスト画像をアップロードしてください")
test_files = files.upload()

for filename in test_files.keys():
    print(f"\n=== {filename} の分析 ===")
    
    # 分析実行
    analysis = detector.analyze(filename)
    
    # 結果表示
    plt.figure(figsize=(8, 6))
    plt.imshow(analysis['image'])
    plt.title(f"{analysis['emoji']} {analysis['status']} (信頼度: {analysis['confidence']:.1%})", 
             fontsize=16)
    plt.axis('off')
    plt.show()
    
    print(f"判定: {analysis['status']}")
    print(f"信頼度: {analysis['confidence']:.1%}")
    print(f"推奨アクション: {analysis['action']}")

## 6. Gradioを使ったWebアプリ化

In [ ]:
# シンプル版Webアプリ
def create_simple_app(detector):
    """
    シンプルで使いやすいWebアプリ
    """
    def process_image(image):
        if image is None:
            return "画像をアップロードしてください"
        
        # 分析実行
        analysis = detector.analyze(image)
        
        # 結果メッセージ
        message = f"""
{analysis['emoji']} **判定結果: {analysis['status']}**

**信頼度**: {analysis['confidence']:.1%}
**推奨アクション**: {analysis['action']}
"""
        return message
    
    # インターフェース作成
    app = gr.Interface(
        fn=process_image,
        inputs=gr.Image(label="検査画像", type="numpy"),
        outputs=gr.Markdown(label="判定結果"),
        title="🔍 傷検出AIシステム（シンプル版）",
        description="部品の画像をアップロードすると、AIが傷の有無を判定します。",
        examples=[],  # サンプル画像があれば追加
        theme=gr.themes.Soft()
    )
    
    return app

# アプリ作成と起動
app = create_simple_app(detector)
app.launch(share=True)

## 7. モデルの評価と改善

In [ ]:
# 混同行列の作成
def evaluate_model(model, test_dataset):
    """
    モデルの詳細な評価
    """
    from sklearn.metrics import confusion_matrix, classification_report
    import seaborn as sns
    
    # 予測と正解を収集
    y_true = []
    y_pred = []
    
    for images, labels in test_dataset:
        predictions = model.predict(images, verbose=0)
        for i in range(len(labels)):
            y_true.append(int(labels[i]))
            y_pred.append(1 if predictions[i] > 0.5 else 0)
    
    # 混同行列
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['不良品', '良品'],
                yticklabels=['不良品', '良品'])
    plt.title('混同行列', fontsize=16)
    plt.ylabel('実際のラベル')
    plt.xlabel('予測ラベル')
    plt.show()
    
    # 詳細レポート
    print("\n📊 分類レポート:")
    print(classification_report(y_true, y_pred, 
                              target_names=['不良品', '良品']))
    
    # 精度計算
    accuracy = sum(1 for t, p in zip(y_true, y_pred) if t == p) / len(y_true)
    print(f"\n✅ 全体精度: {accuracy:.1%}")
    
    return cm, accuracy

# 評価実行（検証データセットで）
# cm, acc = evaluate_model(model, val_ds)

## まとめ

このシンプル版では：

1. **直接的な実装**: TensorFlow/Kerasを使った分かりやすいCNN
2. **カスタマイズ可能**: モデルの構造を自由に変更できる
3. **学習過程が見える**: 精度の向上を確認しながら学習
4. **軽量**: Teachable Machineより小さいモデルサイズ

### 改善のヒント

1. **データを増やす**: 各クラス100枚以上が理想
2. **モデルを調整**: 層を増やしたり、ニューロン数を変更
3. **学習率の調整**: optimizerのlearning_rateを変更
4. **転移学習**: 事前学習済みモデル（MobileNet等）を使用

この実装により、AIの仕組みをより深く理解できます！